# Summary Statistics

In [1]:
import sqlite3

import httpx
import pandas as pd

from litreview import ClinicalTrials

# Make pandas display numbers in non-scientific notation
pd.set_option("display.float_format", lambda x: f"{x:0.1f}")

## Retrieve Integer Fields

In [2]:
API_BASE = "https://clinicaltrials.gov/api/v2"
API_FIELD_VALUES = API_BASE + "/stats/field/values"

response = httpx.get(API_FIELD_VALUES, params={"types": "INTEGER|NUMBER"})
response.raise_for_status()
data = response.json()
integer_fields = [field["piece"] for field in data]
integer_fields

['DispFirstPostYear',
 'DispFirstSubmitYear',
 'EnrollmentCount',
 'EventGroupDeathsNumAffected',
 'EventGroupDeathsNumAtRisk',
 'EventGroupOtherNumAffected',
 'EventGroupOtherNumAtRisk',
 'EventGroupSeriousNumAffected',
 'EventGroupSeriousNumAtRisk',
 'LargeDocSize',
 'LastUpdatePostYear',
 'LastUpdateSubmitYear',
 'NumArmGroupInterventionNames',
 'NumArmGroups',
 'NumAvailIPDs',
 'NumBaselineCategories',
 'NumBaselineClasses',
 'NumBaselineDenoms',
 'NumBaselineGroups',
 'NumBaselineMeasureDenoms',
 'NumBaselineMeasurements',
 'NumBaselineMeasures',
 'NumCentralContacts',
 'NumCollaborators',
 'NumCollaboratorsPlusLead',
 'NumConditionAncestors',
 'NumConditionBrowseBranches',
 'NumConditionBrowseLeafs',
 'NumConditionMeshes',
 'NumConditions',
 'NumDesignWhoMaskeds',
 'NumEventGroups',
 'NumEvents',
 'NumFlowAchievements',
 'NumFlowDropWithdraws',
 'NumFlowGroups',
 'NumFlowMilestones',
 'NumFlowPeriods',
 'NumFlowReasons',
 'NumIPDSharingInfoTypes',
 'NumInterventionAncestors',
 'N

## Load Data from Database

In [6]:
db_connection = sqlite3.connect("../clinical_trials.db")
trials = ClinicalTrials(connection=db_connection, schema_directory="../files/schema.json")

In [12]:
fields = [f for f in integer_fields if f in trials._schema]
data = trials.query(*fields)
pd.DataFrame(data, columns=fields)

,EnrollmentCount,MaximumAge,MinimumAge,NPtrsToThisExpAccNCTId,TargetDuration
0,NaN,120 Years,18 Years,NaN,None
1,40.0,80 Years,50 Years,NaN,None
2,112.0,55 Years,23 Years,NaN,None
3,0.0,21 Years,12 Years,NaN,None
4,212.0,17 Years,13 Years,NaN,None
...,...,...,...,...,...
995,1000.0,30 Years,1 Year,NaN,None
996,440.0,8 Years,None,NaN,None
997,80.0,None,18 Years,NaN,None
998,36.0,60 Years,18 Years,NaN,None
